In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
oldf=pd.read_csv("C:\\Users\\zkyx1\\Desktop\\school\\SRT521\\project\\messages.csv")


In [3]:
oldf['message'] = oldf['message'].str.lower()
# we require our messages to be in lowercase format

In [4]:
oldf.drop_duplicates(inplace=True)
oldf.size
# ok verify message before dropping

8634

In [5]:
oldf.isnull().sum # ok verify message before dropping

<bound method DataFrame.sum of       subject  message  label
0       False    False  False
1        True    False  False
2       False    False  False
3       False    False  False
4       False    False  False
...       ...      ...    ...
2890    False    False  False
2891    False    False  False
2892    False    False  False
2893    False    False  False
2894    False    False  False

[2878 rows x 3 columns]>

In [6]:
df = oldf.dropna()
df.size

8448

In [7]:
df['message'] = df['message'].str.lower()


C:\Users\zkyx1\AppData\Local\Temp\ipykernel_31408\1576429111.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['message'] = df['message'].str.lower()


In [8]:
df['sub_mssg']=df['subject']+df['message']
df.head()


C:\Users\zkyx1\AppData\Local\Temp\ipykernel_31408\1593975471.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sub_mssg']=df['subject']+df['message']


,subject,message,label,sub_mssg
0,job posting - apple-iss research center,content - length : 3386 apple-iss research cen...,0,job posting - apple-iss research centercontent...
2,query : letter frequencies for text identifica...,i am posting this inquiry for sergei atamas ( ...,0,query : letter frequencies for text identifica...
3,risk,a colleague and i are researching the differin...,0,riska colleague and i are researching the diff...
4,request book information,earlier this morning i was on the phone with a...,0,request book informationearlier this morning i...
5,call for abstracts : optimality in syntactic t...,content - length : 4437 call for papers is the...,0,call for abstracts : optimality in syntactic t...


In [9]:
df['len']=df['sub_mssg'].apply(len)


C:\Users\zkyx1\AppData\Local\Temp\ipykernel_31408\2253531254.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['len']=df['sub_mssg'].apply(len)


In [10]:
df.drop('subject',axis=1,inplace=True)


C:\Users\zkyx1\AppData\Local\Temp\ipykernel_31408\1706636209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('subject',axis=1,inplace=True)


In [11]:
import re

def decontract(phrase):
    contractions = {
        "won't": "will not",
        "can't": "can not",
        "n't": " not",
        "'re": " are",
        "'s": " is",
        "'d": " would",
        "'ll": " will",
        "'t": " not",
        "'ve": " have",
        "'m": " am"
    }

    # Replace contractions with their expanded forms
    for contraction, expansion in contractions.items():
        phrase = re.sub(contraction, expansion, phrase)

    return phrase

In [12]:
test_msg = decontract(df['message'][70])
test_msg

'hi , i am new to the list . and since english is not my first language , i apologize for the mistakes you might find below : - ) . i hope that you will not mind if some of my explanations are in french . for my ph . d . semiotics , i am writing a dissertation on discourse and science-fiction and i need some information about the construction of neologisms in french and in english . ( neologisms are useful in sf , specially in the scientific field ! ) in french , neologisms are made in 5 different ways ( i hope these are the right english words ) : derivation ; composition ; imitation ; pure invention ; amalgam . one might also add borrowings from other languages . here are some examples ( sorry , the explanations are in french ) : - derivation : mot derive : " position " peut produire le verbe " positionner " ; - composition : mot compose : si " monotone " est decompose ( " mono " / " tone " ) , on peut recomposer " polytone " ; a partir de " telephone " on peut produire en science-fi

In [13]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Remove stopwords
stopwords = set(['the', 'and', 'is', 'in', 'it', 'to', 'of', 'for', 'this', 'that'])
df['sub_mssg'] = df['sub_mssg'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df['sub_mssg'] = df['sub_mssg'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

# Remove non-ASCII characters
df['sub_mssg'] = df['sub_mssg'].apply(lambda x: ''.join([char if ord(char) < 128 else ' ' for char in x]))

# Remove consecutive duplicates
df['sub_mssg'] = df['sub_mssg'].str.replace(r'(\w)\1+', r'\1')

# Replace repeated characters (e.g., loooove -> love)
df['sub_mssg'] = df['sub_mssg'].str.replace(r'(.)\1+', r'\1')

from nltk.tokenize import word_tokenize
df['sub_mssg'] = df['sub_mssg'].apply(lambda x: ' '.join(word_tokenize(x)))

# Remove short words (length less than 3)
df['sub_mssg'] = df['sub_mssg'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 2]))
# Remove hyphens from the 'sub_mssg' column
df['sub_mssg'] = df['sub_mssg'].str.replace(r'-', ' ')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zkyx1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zkyx1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\zkyx1\AppData\Local\Temp\ipykernel_31408\2867170377.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sub_mssg'] = df['sub_mssg'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))
C:\Users\zkyx1\AppData\Local\Temp\ipykernel_31408\2867170377.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [14]:
df.drop('message',axis=1,inplace=True)
df.isnull().sum()


C:\Users\zkyx1\AppData\Local\Temp\ipykernel_31408\3980139324.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('message',axis=1,inplace=True)


label       0
sub_mssg    0
len         0
dtype: int64

In [15]:
df.head()

,label,sub_mssg,len
0,0,job posting apple iss research centercontent l...,2895
2,0,query letter frequency text identificationi po...,1485
3,0,riska colleague are researching differing degr...,328
4,0,request book informationearlier morning phone ...,1070
5,0,call abstract optimality syntactic theoryconte...,4543


In [16]:
df['len_email']=df['sub_mssg'].apply(len)


C:\Users\zkyx1\AppData\Local\Temp\ipykernel_31408\844218124.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['len_email']=df['sub_mssg'].apply(len)


In [17]:
df.head()

,label,sub_mssg,len,len_email
0,0,job posting apple iss research centercontent l...,2895,2086
2,0,query letter frequency text identificationi po...,1485,1143
3,0,riska colleague are researching differing degr...,328,263
4,0,request book informationearlier morning phone ...,1070,713
5,0,call abstract optimality syntactic theoryconte...,4543,3435


In [24]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold, cross_val_predict, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score


X = df.sub_mssg
Y = df.label


num_folds = 5  
stratified_kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=0)


count_vectorizer = CountVectorizer()
nb_classifier = MultinomialNB()

# Create a pipeline
model_nb = Pipeline([('vectorizer', count_vectorizer), ('classifier', nb_classifier)])

param_grid = {'classifier__alpha': [0.1, 0.5, 1.0, 1.5, 2.0]}
grid_search = GridSearchCV(model_nb, param_grid, cv=stratified_kfold, scoring='accuracy')
grid_search.fit(X, Y)


best_model = grid_search.best_estimator_

# Display the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Use cross_val_predict to get predictions for each fold during cross-validation
y_pred_best_cv = cross_val_predict(best_model, X, Y, cv=stratified_kfold)

# Evaluate performance metrics over the entire dataset
print("Confusion Matrix:\n", confusion_matrix(Y, y_pred_best_cv))
print("Accuracy:", accuracy_score(Y, y_pred_best_cv))
print("Precision:", precision_score(Y, y_pred_best_cv, average='weighted'))
print("Recall:", recall_score(Y, y_pred_best_cv, average='weighted'))



Best Hyperparameters: {'classifier__alpha': 2.0}
Confusion Matrix:
 [[2354    5]
 [   5  452]]
Accuracy: 0.9964488636363636
Precision: 0.9964488636363636
Recall: 0.9964488636363636


In [44]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pickle


# TF= frequency of word in sentence
X = df.sub_mssg
Y = df.label

# Define your model with CountVectorizer for Bag of Words and the best alpha
count_vectorizer = CountVectorizer()
nb_classifier = MultinomialNB(alpha=2.0)
model_nb = Pipeline([('vectorizer', count_vectorizer), ('classifier', nb_classifier)])

# Fit the model on the entire dataset
model_nb.fit(X, Y)

pickle_filename = 'spamdetection_model.pkl'
with open(pickle_filename, 'wb') as model_file:
    pickle.dump(model_nb, model_file)


In [56]:
pickle_filename = 'spamdetection_model.pkl'

with open(pickle_filename, 'rb') as model_file:
    loaded_model = pickle.load(model_file)
user_subject = input("Enter the subject: ")
user_message = input("Enter the message: ")

user_data = [user_subject + ": " + user_message]
predictions = loaded_model.predict(user_data)

if predictions == 0:
    print('not spam')
elif predictions == 1:
    print('spam')

spam
